### Trade Demo

#### Goals:
- Login to the Canada domain.
- Select the dataset.
- Cacluate the sum of total of good imported and exported to Egypt.
- Request for the sum
- Download the results

### Step 1: Login into the Canada domain

In [ ]:
# As a Data Scientist we want to perform some analysis on the trade dataset available on the Canada domain.
# Our goal is to calculate the sum total of the goods imported (imports + re-imports) 
# and exported (exports + re-exports) to Egypt.

import syft as sy

# Let's login into the canada domain
canada_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)

### Step 2: Select the dataset

In [ ]:
# Awesome !!! We're logged into the domain node
# Let's quickly list all the datasets available on the Canada domain

canada_domain_node.datasets

In [ ]:
canada_domain_node.datasets[0]

In [ ]:
feb2020 = canada_domain_node.datasets[0]['feb2020']
feb2020.tags

In [ ]:
# # Let's try to get the whole dataset itself.
feb2020.request(reason="Access whole dataset")

In [ ]:
# Let's see if our request was accepted
feb2020.get()

In [ ]:
# Seems, like the request to download the whole dataset was denied by the Data Owner.

In [ ]:
# Let's continue with the analysis remotely.
# Let's calculate the value of the goods import from Egypt.

value_of_goods_imported_from_egypt = feb2020[(feb2020['Partner'] == 'Egypt') & (feb2020['Trade Flow'] == 'Imports')]['Trade Value (US$)'].sum()
value_of_goods_reimported_from_egypt = feb2020[(feb2020['Partner'] == 'Egypt') & (feb2020['Trade Flow'] == 'Re-imports')]['Trade Value (US$)'].sum()

In [ ]:
value_of_goods_imported_from_egypt.request(reason="Goods value imported from Egypt")

In [ ]:
value_of_goods_reimported_from_egypt.request(reason="Goods value imported from Egypt")

In [ ]:
# wait for data owner to accept/deny request

In [ ]:
imported_goods_value = value_of_goods_imported_from_egypt.get(delete_obj=False)
reimported_goods_value = value_of_goods_reimported_from_egypt.get(delete_obj=False)
print(f"Total trade value($) of the goods Imported from Egypt to Canada: ${imported_goods_value + reimported_goods_value}")

In [ ]:
# Similarly, we can get the get the goods being exported to Egypt.

value_of_goods_exported_to_egypt = feb2020[(feb2020['Partner'] == 'Egypt') & (feb2020['Trade Flow'] == 'Exports') & (feb2020['Trade Flow'] == 'Exports')]['Trade Value (US$)'].sum()
value_of_goods_reexported_to_egypt = feb2020[(feb2020['Partner'] == 'Egypt') & (feb2020['Trade Flow'] == 'Exports') & (feb2020['Trade Flow'] == 'Re-exports')]['Trade Value (US$)'].sum()

In [ ]:
value_of_goods_exported_to_egypt.request(reason="Goods value exported to Egypt")

In [ ]:
value_of_goods_reexported_to_egypt.request(reason="Goods value Re-Exported to Egypt")

In [ ]:
exported_goods_value = value_of_goods_exported_to_egypt.get(delete_obj=False)
reexported_goods_value = value_of_goods_reexported_to_egypt.get(delete_obj=False)
print(f"Total trade value($) of the goods Exported to Egypt from Canada: ${imported_goods_value + reimported_goods_value}")

*Great !!! We successfully, finished our analysis.*